In [5]:
from libdl import DenseLayer, TensorWrapper
import numpy as np

import torch
import torch.nn as nn

In [10]:
dense1 = DenseLayer(10, 10, "test", 10)

data = TensorWrapper(10, 10, 1, 1, True)

result = dense1.forward(data)

torch_data = torch.tensor(data.get_tensor())

linear = nn.Linear(10, 10, True)

torch_out = linear.forward(torch_data)

RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #4 'mat1'

In [4]:
print(data.get_tensor())

[[-0.05588622  0.02034523 -0.06349259  0.04716613 -0.01561551 -0.02589128
  -0.02995522 -0.09423142  0.09743914  0.01826454]
 [-0.0796726  -0.05775554  0.08005107 -0.07883249 -0.04992772  0.04518269
  -0.05739199  0.05246289  0.05968524 -0.02784233]
 [-0.05733096 -0.09742047  0.01828301  0.06241905  0.06483134 -0.0145511
   0.01279037 -0.05627703 -0.07011628  0.05393206]
 [ 0.03858633  0.05421791  0.08256713  0.07692767  0.02035828  0.03960905
  -0.03228517 -0.04801561 -0.04430553  0.03609956]
 [-0.07962348  0.04033824 -0.09431461 -0.0765973  -0.03841372 -0.0907252
  -0.01691233  0.00166758 -0.04300947 -0.07474723]
 [-0.03181649 -0.03518974  0.05438846  0.02916993 -0.01936997 -0.01847341
  -0.04554434  0.02709157 -0.06022967 -0.07676361]
 [ 0.06032676 -0.03403929 -0.09321489  0.05907644  0.04897774 -0.08264273
  -0.06367962 -0.07204129 -0.07999882  0.03399311]
 [-0.0880858  -0.09830531 -0.06259337  0.09774021 -0.03106447  0.02365503
  -0.06339606  0.00204745  0.05990606 -0.07780599]
 [